In [3]:
%pip install py2neo

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 2.8 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from py2neo import Graph

# Connect to Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

# Create a node
graph.run("CREATE (:State {name:'A'})")

(No data)

In [ ]:
from py2neo import Graph

def create_markov_chain_from_matrix(graph, transition_matrix):
    """
    Crée une chaîne de Markov dans Neo4j à partir d'une matrice de transition N×N.
    
    - Chaque état est un nœud `(:State {name: "S0", current: 0})`
    - Les transitions sont des relations `(:State)-[:TO {probability: X}]->(:State)`
    - Seules les transitions avec `probability > 0` sont ajoutées
    
    Paramètres:
    graph: Instance py2neo.Graph (connectée à Neo4j)
    transition_matrix: Liste de listes (matrice NxN avec probabilités)
    """
    
    N = len(transition_matrix)  # Taille de la matrice (NxN)
    
    # Nettoyer les données existantes
    graph.run("MATCH (n) DETACH DELETE n")

    # Créer les nœuds d'état avec `current=0`
    for i in range(N):
        graph.run("CREATE (:State {name: $name, current: 0})", name=f"S{i}")

    # Créer les transitions basées sur la matrice
    for i in range(N):
        for j in range(N):
            probability = transition_matrix[i][j]
            if probability > 0:  # Créer seulement les relations valides
                graph.run(
                    "MATCH (a:State {name: $from_state}), (b:State {name: $to_state}) "
                    "CREATE (a)-[:TO {probability: $prob}]->(b)",
                    from_state=f"S{i}", to_state=f"S{j}", prob=probability
                )

    print(f"Chaîne de Markov avec {N} états créée avec `current=0` sur chaque nœud.")

# Connexion à Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))




Chaîne de Markov avec 4 états créée avec `current=0` sur chaque nœud.


In [15]:
# Exemple de matrice de transition (4x4)
transition_matrix = [
    [0.0, 0.5, 0.5, 0.0],  # S0 -> S1 (0.5), S2 (0.5)
    [0.3, 0.0, 0.7, 0.0],  # S1 -> S0 (0.3), S2 (0.7)
    [0.4, 0.4, 0.0, 0.2],  # S2 -> S0 (0.4), S1 (0.4), S3 (0.2)
    [0.2, 0.3, 0.5, 0.0]   # S3 -> S0 (0.2), S1 (0.3), S2 (0.5)
]

# Créer la chaîne de Markov avec `current=0`
create_markov_chain_from_matrix(graph, transition_matrix)

Chaîne de Markov avec 4 états créée avec `current=0` sur chaque nœud.


In [16]:
from py2neo import Graph

def define_current(graph, node_name):
    """
    Met à jour l'état `current` dans Neo4j.
    
    - Met `current=0` sur tous les nœuds
    - Met `current=1` sur le nœud spécifié
    
    Paramètres:
    graph: Instance py2neo.Graph (connectée à Neo4j)
    node_name: Nom du nœud à activer (ex: "S0")
    """
    
    # Met `current=0` partout
    graph.run("MATCH (s:State) SET s.current = 0")

    # Met `current=1` sur le nœud spécifié
    graph.run("MATCH (s:State {name: $node_name}) SET s.current = 1", node_name=node_name)

    print(f"Le nœud {node_name} a été défini avec `current=1`.")



# Exemple: Définir `current=1` sur S2
define_current(graph, "S2")


Le nœud S2 a été défini avec `current=1`.
